# Data Science Pandas - Joins, Time Series & Rolling Statistics, SQL

## Tasks Today:

1) <b>Pandas</b> <br>
 &nbsp;&nbsp;&nbsp;&nbsp; a) Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - merge() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Breaking Down the Inner Join <br>
 &nbsp;&nbsp;&nbsp;&nbsp; b) Outer Joins <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - Use Case Example <br>
 &nbsp;&nbsp;&nbsp;&nbsp; c) Rolling Statistics and Time Series w/Pandas <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - read_csv() <br>
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - shift() <br>
 2) <b> SQL </b> <br>
      - Adding DataFrame to Database Table
      - Reading SQL Query Result into DataFrame

In [1]:
import pandas as pd
import numpy as np

### Joins <br>
<p>Very often you will have to combine data from several different sources to obtain the actual dataset you need for your exploration or modeling. Pandas draws heavily on SQL in its API design for joins, and if you are familiar with SQL joins, then Pandas will come naturally. Imagine you have two tables, one with customer name and gender, and the other with their purchases:</p>

In [1]:
# Generate some fake data

city_data = {
    1: 'Chicago',
    2: 'Boston'
}

ages = np.random.randint(18,65,100)
city = [city_data[num] for num in np.random.randint(1,6,100)]
customer_id =np.arange(1000,1100)

<p>Let's assume we have a dataset containing the customer ID's cities, and ages for a set of customers. Suppose further, that we have a second table, containing some customer ID's, and a feedback survey rating from those customers. Suppose finally, that you as an analyst are asked the following question: What city tends to give the best feedback for our product? We'll need a join operation to get the answer.</p>

In [ ]:
customer_demographics = pd.DataFrame.from_dict({
    'age': ages,
    'city': city,
    'cusomter_id': customer_id
})

customer_demographics

In [2]:
# Generate some other fake data
customer_id = np.random.randint(1000,1200,100)
customer_rating = np.random.randint(1,11,100)

surveys = pd.DataFrame.from_dict({
    'customer_id': customer_id,
    'customer_rating': customer_rating
})

surveys.head(20)

##### merge()

<p>We can zip the two dataframes up with an inner join, using the Pandas API as follows:</p>

In [ ]:
demo_ratings = customer_demographics.merge(surveys, on = 'customer_id', how='inner')
demo_ratings

In [ ]:
demo_ratings_dropped = demo_ratings.drop_duplicates(subset= 'customer_id').reset_index(drop=True)
demo_ratings_dropped

**.join() method**

In [3]:
# join() is used to combine two DataFrames on the index but not on
# columns whereas merge() is primarily used to specify the columns
# you wanted to join on, this also supports joining on indexes and combination of index and columns

demo_surveys_join = customer_demographics.join(other = surveys.set_index('customer_id'), on = 'customer_id', how = 'inner')
demo_surveys_join

demo_surveys_join_nodups = demo_surveys_join.drop_duplicates(subset = 'customer_id').reset_index(drop=True)
demo_surveys_join_nodups

<p>Now it's a simple groupby operation to reveal the answer:</p>

In [ ]:
city_ages_ratings = demo_ratings_dropped.groupby('city', as_index = False).mean().round(decimals=2)[['city','ages','customer_rating']]
city_ages_ratings

But what if we didn't drop the customer_id duplicates...would we get the same city result??

In [ ]:
city_age_ratings_dups = demo_ratings.groupby('city', as_index = False).mean().round(decimals=2)[['city'.'ages','customer_rating']]
city_age_ratings_dups

What would be the total mean for the US customers?


In [ ]:
city_ratings_mean = demo_ratings_dropped.groupby('city').mean()
city_ratings_mean.loc['Boston']

##### Breaking Down the Inner Join <br>
<p>Let's take a closer look at the actual mechanics here however. There is some subtlety and nuance that is best understood by thinking of an inner join as two steps: a cartesian product, and then a filter. Observe the result of the following inner join:</p>

In [ ]:
d = {
    'A': [1,2],
    'B': ['a','b']
}

df1 = pd.DataFrame.from_dict(d)
df1

In [ ]:
d2 = {
    'A': [1,2],
    'B': ['a','b']
}

df2 = pd.DataFrame.from_dict(d2)
df2


In [ ]:
df1.merge(df2, on='A', how='inner')

<p>Somehow, we ended up with more rows in the result than either of the original tables! This is because we joined on a $\textit{non-unique index}$. The join starts by computing all the possible combinations of rows, and then filters them based on the condition. Because of the duplicates, multiple combinations of rows make it through!</p>

### Outer Joins <br>
<p>An $\textit{outer join}$ produces all of the rows from the left table, and joins whatever rows it can to the right table, filling anywhere it cant with NaN's or None values. Below is a brief example, and then we will discuss a possible use case for this type of join.</p>

In [ ]:
df1.merge(df2, on = 'A', how = 'outer')

In [4]:
# LEFT JOIN MERGE
df1.merge(df2, on = 'A', how = 'left')

In [5]:
# RIGHT JOIN MERGE
df1.merge(df2, on = 'A', how = 'right')

##### Use Case Example

<p>As an example use case, let's imagine we have again the demographics and survey response tables from above:</p>

In [ ]:
customer_demographics.head()

In [ ]:
surveys.head()

<p>Now, you are asked as an analyst to compare the demographics of survey responders to the demographics of all your customers combined.</p>

In [ ]:
demo_ratings_outer = customer_demographics.merge(surveys, on='customer_id', how='outer')

demo_ratings_outer['customer_rating'] = demo_ratings_outer['customer_rating'].fillna('N/A')

demo_ratings_outer

In [7]:
# Count all survey responders against the total number of customers we have 
# (Specifically the customers that haven't taken the survey yet)
demo_size = demo_ratings_outer.groupby(['city', 'customer_rating']).size()
demo_size

demo_count = demo_ratings_outer.groupby(['city', 'customer_rating']).count()
demo_count
# Producing similar result with .count()


In [8]:
# average age of people that answered the survey
survey_answered = demo_ratings_outer[demo_ratings_outer['customer_rating'] != 'N/A']

round(survey_answered['ages'].mean())

# demo_ratings_outer['customer_rating'] != 'N/A'


In [9]:
# find the average age of people that did not answer the survey


### Rolling Statistics and Time Series with Pandas

##### read_csv()

In [ ]:
weather = pd.read_csv('files/chiweather.csv', sep = ';')

In [ ]:
weather.head(10)

In [ ]:
weather_update = weather.groupby(['Year','Month','Day']).mean().reset_index()

In [ ]:
weather_update.head(10)

##### shift() <br>
<p>Shift index by desired number of periods with an optional time freq</p>

In [10]:
# Print all keys for easier use
print(weather_update.keys())

weather_update['last_temp'] = weather_update['Temperature [2 m above gnd]'].shift(1)

In [ ]:
weather_update

In [ ]:
weather_update['difference']=abs(weather_update['last_temp']-weather_update['Temperature [2 m above gnd]'])
weather_update

In [11]:
# create a new column that takes the difference in temperature from the previous day as an absolute value


In [12]:
# Pip install for both psycopg2 and SQLAlchemy
import sys
!{sys.executable} -m pip install psycopg2-binary SQLAlchemy # the -binary is for mac users

In [14]:
# databaselanguage(postgresql)://user:password@url:port/database_name



In [13]:
# Writing a DataFrame to PostGreSQL database 
# from sqlalchemy.types import Integer, Text,String,DateTime,Float
